In [21]:
import numpy as np
import h5py
from scipy.signal import butter, filtfilt, iirnotch

## Data Preprocess

### read .mat data

In [11]:
with h5py.File('./Dataset/increasingforce1.mat', 'r') as file:
    # List all groups
    print("Keys: %s" % file.keys())
    # Get the data
    increase_grid_one = file['grid_crds'][()].T
    increase_mat_one = file['out_mat'][()].T
    
with h5py.File('./Dataset/increasingforce2.mat', 'r') as file:
    # List all groups
    print("Keys: %s" % file.keys())
    # Get the data
    increase_grid_two = file['grid_crds'][()].T
    increase_mat_two = file['out_mat'][()].T
    
with h5py.File('./Dataset/steadyforce.mat', 'r') as file:
    # List all groups
    print("Keys: %s" % file.keys())
    # Get the data
    steady_grid = file['grid_crds'][()].T
    steady_mat = file['out_mat'][()].T

Keys: <KeysViewHDF5 ['grid_crds', 'out_mat']>
Keys: <KeysViewHDF5 ['grid_crds', 'out_mat']>
Keys: <KeysViewHDF5 ['grid_crds', 'out_mat']>


### filter out noise using filt_GRID

In [17]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    b, a = butter(order, [lowcut/ nyq, highcut/ nyq], btype='band', analog=False)
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y
def notch(notch_freq, samp_freq, quality_factor=30):
    b, a = iirnotch(notch_freq, quality_factor, samp_freq)
    return b, a
def notch_filter(data, notch_fs, fs, q=30):
    b, a = notch(notch_fs, fs, q)
    y = filtfilt(b, a, data)
    return y
def filt_GRID(data, lowcut=20, highcut=500, fs=4000, order=3, notch_fs = 50, notch_q = 30):
    filt_out = np.zeros_like(data)
    for i in range(data.shape[0]):
        filt_out[i,:] = notch_filter(butter_bandpass_filter(data[i,:], lowcut, highcut, fs, order=order), notch_fs, fs, notch_q)
    return filt_out

In [24]:
increase_mat_one = filt_GRID(increase_mat_one)
increase_mat_two = filt_GRID(increase_mat_two)
steady_mat = filt_GRID(steady_mat)

In [25]:
steady_mat.shape

(64, 73214)

In [15]:
steady_mat.shape

(64, 73214)

In [ ]:
def detect_spikes(data):
    # Implement spike detection logic
    pass

def sort_spikes(spikes):
    # Implement spike sorting logic, potentially using clustering or PCA
    pass

def track_motor_units(sorted_spikes):
    # Implement motor unit tracking logic
    pass

def motor_unit_decomposition(hdemg_data):
    # Preprocess the data
    filtered_data = filt_GRID(hdemg_data)
    
    # Detect spikes
    spikes = detect_spikes(filtered_data)
    
    # Sort spikes into potential motor units
    sorted_spikes = sort_spikes(spikes)
    
    # Track motor units across time and channels
    motor_units = track_motor_units(sorted_spikes)
    
    # Validate and refine motor unit detection
    validated_motor_units = validate_motor_units(motor_units)
    
    return validated_motor_units

# Simulated or actual HDEMG data would be processed as follows:
hdemg_data_array1 = # ... load data for array 1
hdemg_data_array2 = # ... load data for array 2
hdemg_data_array3 = # ... load data for array 3

# Decompose motor units for each data array
mus_array1 = motor_unit_decomposition(hdemg_data_array1)
mus_array2 = motor_unit_decomposition(hdemg_data_array2)
mus_array3 = motor_unit_decomposition(hdemg_data_array3)